# Week 4 Exercise (group): Exploratory Data Analysis on Social Media Data

- Lori Lou
- Nayeon Bae
- fai slavianto 
- Ruiya Sun

## 1. Import necessary packages

In [14]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import nltk
import emoji

In [5]:
!pip install emoji

  Using cached emoji-2.2.0-py3-none-any.whl


In [6]:
!pip install nltk

  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
  Using cached regex-2023.5.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (769 kB)
  Using cached click-8.1.3-py3-none-any.whl (96 kB)


post_text## 2. Read the data

The data is called `tweets.csv` in the same folder. More information about the data see [here](https://www.kaggle.com/datasets/infamouscoder/mental-health-social-media)

The main column you will be working with is `post_text`

In [15]:
df = pd.read_csv("tweets.csv")
df
# explore the data characteristic using `df.describe()` or `df.info()`

,Unnamed: 0,post_id,post_created,post_text,user_id,followers,friends,favourites,statuses,retweets,label
0,0,637894677824413696,Sun Aug 30 07:48:37 +0000 2015,It's just over 2 years since I was diagnosed w...,1013187241,84,211,251,837,0,1
1,1,637890384576778240,Sun Aug 30 07:31:33 +0000 2015,"It's Sunday, I need a break, so I'm planning t...",1013187241,84,211,251,837,1,1
2,2,637749345908051968,Sat Aug 29 22:11:07 +0000 2015,Awake but tired. I need to sleep but my brain ...,1013187241,84,211,251,837,0,1
3,3,637696421077123073,Sat Aug 29 18:40:49 +0000 2015,RT @SewHQ: #Retro bears make perfect gifts and...,1013187241,84,211,251,837,2,1
4,4,637696327485366272,Sat Aug 29 18:40:26 +0000 2015,It’s hard to say whether packing lists are mak...,1013187241,84,211,251,837,1,1
...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,819336825231773698,Thu Jan 12 00:14:56 +0000 2017,A day without sunshine is like night.,1169875706,442,230,7,1063601,0,0
19996,19996,819334654260080640,Thu Jan 12 00:06:18 +0000 2017,"Boren's Laws: (1) When in charge, ponder. (2) ...",1169875706,442,230,7,1063601,0,0
19997,19997,819334503042871297,Thu Jan 12 00:05:42 +0000 2017,The flow chart is a most thoroughly oversold p...,1169875706,442,230,7,1063601,0,0
19998,19998,819334419374899200,Thu Jan 12 00:05:22 +0000 2017,"Ships are safe in harbor, but they were never ...",1169875706,442,230,7,1063601,0,0


## 3. Extract emojis

Use `emoji` package to extract emojis and put them into a new column called `emojis`

In [41]:
def extract_emojis(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "]+", flags=re.UNICODE)
    return emoji_pattern.findall(text)

In [42]:
# define the function
emojis = df['post_text'].apply(extract_emojis)
emojis
# apply the function to your dataframe

0        []
1        []
2        []
3        []
4        []
         ..
19995    []
19996    []
19997    []
19998    []
19999    []
Name: post_text, Length: 20000, dtype: object

## 4. Text Cleaning using Regular Expressions 

1. Remove URLs
2. Remove mentions
3. Remove hashtags
4. Remove special characters
5. Remove extra space

Code can be found in [week 6 lecture 1](https://github.com/yibeichan/COMM160DS/blob/main/week_6/lecture_part1.ipynb) section `4.4 All-in-One`

Perform the analysis and save the results into a new column.

In [16]:
# define the function
def clean_text(text):
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)  # remove URLs
    text = re.sub(r'@\w+', '', text)  # remove mentions
    text = re.sub(r'#\w+', '', text) # remove hashtag
    text = re.sub(r'\W', ' ', text)  # remove special characters
    text = re.sub(r'\s+', ' ', text)  # remove extra spaces
    return text.strip()
df['new'] = df['post_text'].apply(clean_text)
# apply the function to your dataframe

## 5. Collocation Analysis

Choose one analysis from (1)Sentiment Analysis, (2)N-grams and Phrase Analysis, (3)Collocation Analysis, (4)Part-of-Speech Tagging, (5)Named Entity Recognition, and (6)Dependency Parsing.

Perform the analysis and save the results into a new column.

In [7]:
nltk.download('stopwords')
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder
from nltk.corpus import stopwords
import string

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
# write your code here
def find_collocations(text, num_collocations=10):
    tokens = nltk.word_tokenize(text)
    bigram_measures = BigramAssocMeasures()
    finder = BigramCollocationFinder.from_words(tokens)
    # Apply a frequency filter to keep only bigrams that appear at least twice
    finder.apply_freq_filter(2)
    # Apply a word filter to exclude bigrams containing stopwords or punctuations
    finder.apply_word_filter(lambda w: w.lower() in stopwords.words('english') or w in string.punctuation)
    # Return the top num_collocations bigrams with the highest PMI scores
    return finder.nbest(bigram_measures.pmi, num_collocations)

In [10]:
collocations = find_collocations.(new)
collocations

SyntaxError: invalid syntax (2899380874.py, line 1)

## 6. N-grams and Phrase Analysis

Choose another analysis from (1)Sentiment Analysis, (2)N-grams and Phrase Analysis, (3)Collocation Analysis, (4)Part-of-Speech Tagging, (5)Named Entity Recognition, and (6)Dependency Parsing.

Perform the analysis and save the results into a new column.

In [11]:
# write your code here
from nltk import ngrams

In [19]:
def generate_ngrams(text, n):
    tokens = text.split()
    return list(ngrams(tokens, n))

In [21]:
df['n_grams'] = df['post_text'].apply(lambda x: generate_ngrams(x, n=2))
df['n_grams']

0        [(It's, just), (just, over), (over, 2), (2, ye...
1        [(It's, Sunday,), (Sunday,, I), (I, need), (ne...
2        [(Awake, but), (but, tired.), (tired., I), (I,...
3        [(RT, @SewHQ:), (@SewHQ:, #Retro), (#Retro, be...
4        [(It’s, hard), (hard, to), (to, say), (say, wh...
                               ...                        
19995    [(A, day), (day, without), (without, sunshine)...
19996    [(Boren's, Laws:), (Laws:, (1)), ((1), When), ...
19997    [(The, flow), (flow, chart), (chart, is), (is,...
19998    [(Ships, are), (are, safe), (safe, in), (in, h...
19999    [(Black, holes), (holes, are), (are, where), (...
Name: n_grams, Length: 20000, dtype: object

## 7. Push Your Results to GitHub

As you did in previous weeks:
1. `git status`
2. `git add`
3. `git commit -m "type your message here"`
4. `git push`

If you can't push it to GitHub, it's okay to manually uploaded it.